# [Effective TensorFlow for Non-Experts (Google IO '17)](http://youtu.be/?v=5DknTFbcGVM)
c:\Users\hcche\Documents\ML YouTube and Moocs\Machine Learning\Misc of ML and AI\Inception slim models API\Effective TensorFlow for Non-Experts (Google I_O '17)-5DknTFbcGVM.mp4                      
# [Estimator demo using Automobile dataset](https://gist.github.com/martinwicke/6838c23abdc53e6bcda36ed9f40cff39)
[Jupyter Notebook martinwicke/automobile.ipynb](https://goo.gl/0OgXiL)




\[x\] 2018.1.3 21:02 沒看懂這個 demo 是要幹嘛? --> 從訓練資料來觀察，看它的 features 跟 labels 就可以猜得出來 --> training_data = df\[:160\]; training_label = training_data.pop('price') 原來是要猜價錢，哈！see also the video @ 05:30 

In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [2]:
# First thing to do: Download https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

In [3]:
# We're using pandas to read the CSV file. This is easy for small datasets, but for large and complex datasets,
# tensorflow parsing and processing functions are more powerful.
import pandas as pd
import numpy as np

In [4]:
# The CSV file does not have a header, so we have to fill in column names.
names = [
    'symboling', 
    'normalized-losses', 
    'make', 
    'fuel-type', 
    'aspiration',
    'num-of-doors',
    'body-style',
    'drive-wheels',
    'engine-location',
    'wheel-base',
    'length',
    'width',
    'height',
    'curb-weight',
    'engine-type',
    'num-of-cylinders',
    'engine-size',
    'fuel-system',
    'bore',
    'stroke',
    'compression-ratio',
    'horsepower',
    'peak-rpm',
    'city-mpg',
    'highway-mpg',
    'price',
]

# We also have to specify dtypes.
dtypes = {
    'symboling': np.int32, 
    'normalized-losses': np.float32, 
    'make': str, 
    'fuel-type': str, 
    'aspiration': str,
    'num-of-doors': str,
    'body-style': str,
    'drive-wheels': str,
    'engine-location': str,
    'wheel-base': np.float32,
    'length': np.float32,
    'width': np.float32,
    'height': np.float32,
    'curb-weight': np.float32,
    'engine-type': str,
    'num-of-cylinders': str,
    'engine-size': np.float32,
    'fuel-system': str,
    'bore': np.float32,
    'stroke': np.float32,
    'compression-ratio': np.float32,
    'horsepower': np.float32,
    'peak-rpm': np.float32,
    'city-mpg': np.float32,
    'highway-mpg': np.float32,
    'price': np.float32,    
}

In [5]:
# Read the file.
df = pd.read_csv('imports-85.data', names=names, dtype=dtypes, na_values='?')

In [6]:
df

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.599998,...,130.0,mpfi,3.47,2.68,9.00,111.0,5000.0,21.0,27.0,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.599998,...,130.0,mpfi,3.47,2.68,9.00,111.0,5000.0,21.0,27.0,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.500000,...,152.0,mpfi,2.68,3.47,9.00,154.0,5000.0,19.0,26.0,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.800003,...,109.0,mpfi,3.19,3.40,10.00,102.0,5500.0,24.0,30.0,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.400002,...,136.0,mpfi,3.19,3.40,8.00,115.0,5500.0,18.0,22.0,17450.0
5,2,NaN,audi,gas,std,two,sedan,fwd,front,99.800003,...,136.0,mpfi,3.19,3.40,8.50,110.0,5500.0,19.0,25.0,15250.0
6,1,158.0,audi,gas,std,four,sedan,fwd,front,105.800003,...,136.0,mpfi,3.19,3.40,8.50,110.0,5500.0,19.0,25.0,17710.0
7,1,NaN,audi,gas,std,four,wagon,fwd,front,105.800003,...,136.0,mpfi,3.19,3.40,8.50,110.0,5500.0,19.0,25.0,18920.0
8,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.800003,...,131.0,mpfi,3.13,3.40,8.30,140.0,5500.0,17.0,20.0,23875.0
9,0,NaN,audi,gas,turbo,two,hatchback,4wd,front,99.500000,...,131.0,mpfi,3.13,3.40,7.00,160.0,5500.0,16.0,22.0,NaN


In [7]:
# Some rows don't have price data, we can't use those.
df = df.dropna(axis='rows', how='any', subset=['price'])

In [8]:
df

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.599998,...,130.0,mpfi,3.47,2.68,9.00,111.0,5000.0,21.0,27.0,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.599998,...,130.0,mpfi,3.47,2.68,9.00,111.0,5000.0,21.0,27.0,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.500000,...,152.0,mpfi,2.68,3.47,9.00,154.0,5000.0,19.0,26.0,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.800003,...,109.0,mpfi,3.19,3.40,10.00,102.0,5500.0,24.0,30.0,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.400002,...,136.0,mpfi,3.19,3.40,8.00,115.0,5500.0,18.0,22.0,17450.0
5,2,NaN,audi,gas,std,two,sedan,fwd,front,99.800003,...,136.0,mpfi,3.19,3.40,8.50,110.0,5500.0,19.0,25.0,15250.0
6,1,158.0,audi,gas,std,four,sedan,fwd,front,105.800003,...,136.0,mpfi,3.19,3.40,8.50,110.0,5500.0,19.0,25.0,17710.0
7,1,NaN,audi,gas,std,four,wagon,fwd,front,105.800003,...,136.0,mpfi,3.19,3.40,8.50,110.0,5500.0,19.0,25.0,18920.0
8,1,158.0,audi,gas,turbo,four,sedan,fwd,front,105.800003,...,131.0,mpfi,3.13,3.40,8.30,140.0,5500.0,17.0,20.0,23875.0
10,2,192.0,bmw,gas,std,two,sedan,rwd,front,101.199997,...,108.0,mpfi,3.50,2.80,8.80,101.0,5800.0,23.0,29.0,16430.0


    媽呀！下面這 cell -- Pandas 太厲害了， df 竟然可以這樣操作。
    OK __main__ :> float_columns constant float_columns 
        // ( -- list ) df float_columns
    OK float_columns . cr
    ['normalized-losses', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

In [15]:
# Fill missing values in continuous columns with zeros instead of NaN.
float_columns = [k for k,v in dtypes.items() if v == np.float32]
df[float_columns] = df[float_columns].fillna(value=0., axis='columns')
# Fill missing values in continuous columns with '' instead of NaN (NaN mixed with strings is very bad for us).
string_columns = [k for k,v in dtypes.items() if v == str]
df[string_columns] = df[string_columns].fillna(value='', axis='columns')

c:\users\hcche\appdata\local\programs\python\python36\lib\site-packages\pandas\core\frame.py:2450: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [16]:
# Split the data into a training set and an eval set.
training_data = df[:160]
eval_data = df[160:]

# Separate input features from labels
training_label = training_data.pop('price')
eval_label = eval_data.pop('price')

In [17]:
# Now we can start using some TensorFlow.
import tensorflow as tf
print('please make sure that version >= 1.2:')
print(tf.__version__)

please make sure that version >= 1.2:
1.4.0


    影片中 @ 12:38 介紹到下面的 pandas_input_fn() 之外，還有很多選擇。查看看：
    dir(tf.estimator.inputs)
    [...snip...,  '_allowed_symbols', 'numpy_input_fn', 'pandas_input_fn']
    老師還提到的 generator_input_fn 不知是哪來的，表示不限於此。

In [18]:
# Make input function for training: 
#   num_epochs=None -> will cycle through input data forever
#   shuffle=True -> randomize order of input data
training_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=training_data, y=training_label, 
    batch_size=64, shuffle=True, num_epochs=None)

# Make input function for evaluation:
#   shuffle=False -> do not randomize input data
eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=eval_data, y=eval_label, batch_size=64, shuffle=False)

    定義好 input_fn 之後，以下要描述 feature columns. 不同的 classifier / regressor 所能接受的 feature column 不同。以下要先為 linearRegressor 泡製合適的 feature column，稍後改用 DNNRegressor 又要另外調製一番。

In [19]:
# Describe how the model should interpret the inputs. 
# The names of the feature columns have to match the names
# of the series in the dataframe.

# 影片中 @ 06:00 介紹到這裡，有三種 type : numeric, categorical with hash bucket, and with vocabulary list

symboling = tf.feature_column.numeric_column('symboling')
normalized_losses = tf.feature_column.numeric_column('normalized-losses')
make = tf.feature_column.categorical_column_with_hash_bucket('make', 50)
fuel_type = tf.feature_column.categorical_column_with_vocabulary_list('fuel-type', vocabulary_list=['diesel', 'gas'])
aspiration = tf.feature_column.categorical_column_with_vocabulary_list('aspiration', vocabulary_list=['std', 'turbo'])
num_of_doors = tf.feature_column.categorical_column_with_vocabulary_list('num-of-doors', vocabulary_list=['two', 'four'])
body_style = tf.feature_column.categorical_column_with_vocabulary_list('body-style', vocabulary_list=['hardtop', 'wagon', 'sedan', 'hatchback', 'convertible'])
drive_wheels = tf.feature_column.categorical_column_with_vocabulary_list('drive-wheels', vocabulary_list=['4wd', 'rwd', 'fwd'])
engine_location = tf.feature_column.categorical_column_with_vocabulary_list('engine-location', vocabulary_list=['front', 'rear'])
wheel_base = tf.feature_column.numeric_column('wheel-base')
length = tf.feature_column.numeric_column('length')
width = tf.feature_column.numeric_column('width')
height = tf.feature_column.numeric_column('height')
curb_weight = tf.feature_column.numeric_column('curb-weight')
engine_type = tf.feature_column.categorical_column_with_vocabulary_list('engine-type', ['dohc', 'dohcv', 'l', 'ohc', 'ohcf', 'ohcv', 'rotor'])
num_of_cylinders = tf.feature_column.categorical_column_with_vocabulary_list('num-of-cylinders', ['eight', 'five', 'four', 'six', 'three', 'twelve', 'two'])
engine_size = tf.feature_column.numeric_column('engine-size')
fuel_system = tf.feature_column.categorical_column_with_vocabulary_list('fuel-system', ['1bbl', '2bbl', '4bbl', 'idi', 'mfi', 'mpfi', 'spdi', 'spfi'])
bore = tf.feature_column.numeric_column('bore')
stroke = tf.feature_column.numeric_column('stroke')
compression_ratio = tf.feature_column.numeric_column('compression-ratio')
horsepower = tf.feature_column.numeric_column('horsepower')
peak_rpm = tf.feature_column.numeric_column('peak-rpm')
city_mpg = tf.feature_column.numeric_column('city-mpg')
highway_mpg = tf.feature_column.numeric_column('highway-mpg')

In [20]:
linear_features = [symboling, normalized_losses, make, fuel_type, aspiration, num_of_doors,
                   body_style, drive_wheels, engine_location, wheel_base, length, width,
                   height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system,
                   bore, stroke, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg]

影片中 @ 12:53 也說 Classifier 有很多，提到的 TensorForest、KMeans 在 tf.contrib.learn 裡沒有，表示不限於此。Ynote "[筆記] TensorFlow high-level APIs" 查出一堆 classifier, Estimator, Regressor 都不知怎麼區分的？ [Stackoverflow 上有人問了](https://stackoverflow.com/questions/43401721/difference-between-dnnlinearcombinedestimator-regressor-classifier-tensorflo) 也沒人回答 --> 我知道了 classifier 分類貓、狗，而 regressor 擬合出一個數字 -- 明顯不同。Estimator 是個 builder API. 本範例以及 WKS WH300D 這類問題就得用 Estimator 因為它的 input function 能處理各種 categorical columns. 如果不用 Estimator 真不知該如何帶進 TensorFlow。

In [21]:
regressor = tf.contrib.learn.LinearRegressor(feature_columns=linear_features)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002B9CF464CC0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'C:\\Users\\hcche\\AppData\\Local\\Temp\\tmpgyz9zuy0'}


In [22]:
regressor.fit(input_fn=training_input_fn, steps=10000)

Instructions for updating:
Please switch to tf.train.get_global_step
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\hcche\AppData\Local\Temp\tmpgyz9zuy0\model.ckpt.
INFO:tensorflow:loss = 2.26154e+08, step = 1
INFO:tensorflow:global_step/sec: 141.139
INFO:tensorflow:loss = 1.04599e+08, step = 101 (0.715 sec)
INFO:tensorflow:global_step/sec: 216.884
INFO:tensorflow:loss = 8.73857e+07, step = 201 (0.461 sec)
INFO:tensorflow:global_step/sec: 181.178
INFO:tensorflow:loss = 3.60493e+07, step = 301 (0.555 sec)
INFO:tensorflow:global_step/sec: 175.604
INFO:tensorflow:loss = 4.00759e+07, step = 401 (0.566 sec)
INFO:tensorflow:global_step/sec: 157.347
INFO:tensorflow:loss = 5.69015e+07, step = 501 (0.636 sec)
INFO:tensorflow:global_step/sec: 191.412
INFO:tensorflow:loss = 4.30927e+07, step = 601 (0.521 sec)
INFO:tensorflow:global_step/sec: 141.116
INFO:tensorflow:loss = 2.46088e+07, step = 701 (0.714 sec)
INFO:tensorflow:global_step/sec: 191.0

INFO:tensorflow:global_step/sec: 132.331
INFO:tensorflow:loss = 1.88338e+07, step = 7701 (0.759 sec)
INFO:tensorflow:global_step/sec: 144.222
INFO:tensorflow:loss = 3.16418e+07, step = 7801 (0.692 sec)
INFO:tensorflow:global_step/sec: 169.117
INFO:tensorflow:loss = 3.06328e+07, step = 7901 (0.592 sec)
INFO:tensorflow:global_step/sec: 134.839
INFO:tensorflow:loss = 2.44862e+07, step = 8001 (0.740 sec)
INFO:tensorflow:global_step/sec: 190.412
INFO:tensorflow:loss = 1.26047e+07, step = 8101 (0.527 sec)
INFO:tensorflow:global_step/sec: 160.602
INFO:tensorflow:loss = 1.94225e+07, step = 8201 (0.622 sec)
INFO:tensorflow:global_step/sec: 202.782
INFO:tensorflow:loss = 2.32412e+07, step = 8301 (0.491 sec)
INFO:tensorflow:global_step/sec: 191.122
INFO:tensorflow:loss = 1.85889e+07, step = 8401 (0.529 sec)
INFO:tensorflow:global_step/sec: 148.241
INFO:tensorflow:loss = 2.06353e+07, step = 8501 (0.680 sec)
INFO:tensorflow:global_step/sec: 130.053
INFO:tensorflow:loss = 1.2975e+07, step = 8601 (0.

LinearRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000002B9CE6F7F60>, 'feature_columns': [_NumericColumn(key='symboling', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _NumericColumn(key='normalized-losses', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), _HashedCategoricalColumn(key='make', hash_bucket_size=50, dtype=tf.string), _VocabularyListCategoricalColumn(key='fuel-type', vocabulary_list=('diesel', 'gas'), dtype=tf.string, default_value=-1, num_oov_buckets=0), _VocabularyListCategoricalColumn(key='aspiration', vocabulary_list=('std', 'turbo'), dtype=tf.string, default_value=-1, num_oov_buckets=0), _VocabularyListCategoricalColumn(key='num-of-doors', vocabulary_list=('two', 'four'), dtype=tf.string, default_value=-1, num_oov_buckets=0), _VocabularyListCategoricalColumn(key='body-style', vocabulary_list=('hardtop', 'wagon', 'sedan', 'hatchback', 'convertible'), dtype=tf.

In [23]:
regressor.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Starting evaluation at 2018-03-08-07:51:13
INFO:tensorflow:Restoring parameters from C:\Users\hcche\AppData\Local\Temp\tmpgyz9zuy0\model.ckpt-10000
INFO:tensorflow:Finished evaluation at 2018-03-08-07:51:14
INFO:tensorflow:Saving dict for global step 10000: global_step = 10000, loss = 8.22575e+06


{'global_step': 10000, 'loss': 8225754.5}

    開始準備改玩 DNNRegressor. 影片 @13:30 說明到這裡，但他投影片用了 embedding_column() 這樣的描述性的虛擬 function 來代表，指的是實際上的 indicator_column() 如下。

In [ ]:
dnn_features = [
    #numerical features 直接沿用上面描述好的 feature column 性質
    symboling, normalized_losses, wheel_base, length, width, height, curb_weight, engine_size,
    bore, stroke, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg,    
    # densify categorical features: 字串形式的要這樣描述，比上面的簡便。
    tf.feature_column.indicator_column(make),
    tf.feature_column.indicator_column(fuel_type),
    tf.feature_column.indicator_column(aspiration),
    tf.feature_column.indicator_column(num_of_doors),
    tf.feature_column.indicator_column(body_style),
    tf.feature_column.indicator_column(drive_wheels), 
    tf.feature_column.indicator_column(engine_location),
    tf.feature_column.indicator_column(engine_type),
    tf.feature_column.indicator_column(num_of_cylinders),
    tf.feature_column.indicator_column(fuel_system),
]

    影片 @ 14:10 解釋說 hidden_units=[50,30,10] 是以個別的 layer size 設定 DNNRegressor 用 3 layers 的 neural network.

In [ ]:
dnnregressor = tf.contrib.learn.DNNRegressor(
    feature_columns=dnn_features, hidden_units=[50, 30, 10])

In [ ]:
dnnregressor.fit(input_fn=training_input_fn, steps=10000)

In [ ]:
dnnregressor.evaluate(input_fn=eval_input_fn)

In [ ]:
def experiment_fn(run_config, params):
  # This function makes an Experiment, containing an Estimator 
  # and inputs for training and evaluation.
  # You can use params and config here to customize the Estimator 
  # depending on the cluster or to use
  # hyperparameter tuning.

  # Collect information for training
  return tf.contrib.learn.Experiment(
      estimator=tf.contrib.learn.LinearRegressor(
          feature_columns=linear_features, config=run_config),
      train_input_fn=training_input_fn,
      train_steps=10000,
      eval_input_fn=eval_input_fn)

In [ ]:
import shutil
shutil.rmtree("/tmp/output_dir", ignore_errors=True)
tf.contrib.learn.learn_runner.run(experiment_fn, run_config=tf.contrib.learn.RunConfig(model_dir="/tmp/output_dir"))

In [ ]:
import peforth

    __main__ :> df constant df // ( -- dataframe ) 
    __main__ :> pd constant pd // ( -- Pandas )

    df :> [:3] . cr
       symboling  normalized-losses         make fuel-type aspiration  \
    0          3                0.0  alfa-romero       gas        std   
    1          3                0.0  alfa-romero       gas        std   
    2          1                0.0  alfa-romero       gas        std   

      num-of-doors   body-style drive-wheels engine-location  wheel-base   ...     \
    0          two  convertible          rwd           front   88.599998   ...      
    1          two  convertible          rwd           front   88.599998   ...      
    2          two    hatchback          rwd           front   94.500000   ...      

       engine-size  fuel-system  bore  stroke compression-ratio horsepower  \
    0        130.0         mpfi  3.47    2.68               9.0      111.0   
    1        130.0         mpfi  3.47    2.68               9.0      111.0   
    2        152.0         mpfi  2.68    3.47               9.0      154.0   

       peak-rpm city-mpg  highway-mpg    price  
    0    5000.0     21.0         27.0  13495.0  
    1    5000.0     21.0         27.0  16500.0  
    2    5000.0     19.0         26.0  16500.0  

    [3 rows x 26 columns]

    \ Pandas 的 DataFrame 功能超多
    OK df dir . cr
    [...snip..., 'abs', 'add', 'add_prefix', 'add_suffix', 'agg', 'aggregate', 'align', 'all', 'any', 'append', 'apply', 'applymap', 'as_blocks', 'as_matrix', 'asfreq', 'asof', 'aspiration', 'assign', 'astype', 'at', 'at_time', 'axes', 'between_time', 'bfill', 'blocks', 'bool', 'bore', 'boxplot', 'clip', 'clip_lower', 'clip_upper', 'columns', 'combine', 'combine_first', 'compound', 'consolidate', 'convert_objects', 'copy', 'corr', 'corrwith', 'count', 'cov', 'cummax', 'cummin', 'cumprod', 'cumsum', 'describe', 'diff', 'div', 'divide', 'dot', 'drop', 'drop_duplicates', 'dropna', 'dtypes', 'duplicated', 'empty', 'eq', 'equals', 'eval', 'ewm', 'expanding', 'ffill', 'fillna', 'filter', 'first', 'first_valid_index', 'floordiv', 'from_csv', 'from_dict', 'from_items', 'from_records', 'ftypes', 'ge', 'get', 'get_dtype_counts', 'get_ftype_counts', 'get_value', 'get_values', 'groupby', 'gt', 'head', 'height', 'hist', 'horsepower', 'iat', 'idxmax', 'idxmin', 'iloc', 'index', 'info', 'insert', 'interpolate', 'is_copy', 'isin', 'isnull', 'items', 'iteritems', 'iterrows', 'itertuples', 'ix', 'join', 'keys', 'kurt', 'kurtosis', 'last', 'last_valid_index', 'le', 'length', 'loc', 'lookup', 'lt', 'mad', 'make', 'mask', 'max', 'mean', 'median', 'melt', 'memory_usage', 'merge', 'min', 'mod', 'mode', 'mul', 'multiply', 'ndim', 'ne', 'nlargest', 'notnull', 'nsmallest', 'nunique', 'pct_change', 'pipe', 'pivot', 'pivot_table', 'plot', 'pop', 'pow', 'price', 'prod', 'product', 'quantile', 'query', 'radd', 'rank', 'rdiv', 'reindex', 'reindex_axis', 'reindex_like', 'rename', 'rename_axis', 'reorder_levels', 'replace', 'resample', 'reset_index', 'rfloordiv', 'rmod', 'rmul', 'rolling', 'round', 'rpow', 'rsub', 'rtruediv', 'sample', 'select', 'select_dtypes', 'sem', 'set_axis', 'set_index', 'set_value', 'shape', 'shift', 'size', 'skew', 'slice_shift', 'sort_index', 'sort_values', 'sortlevel', 'squeeze', 'stack', 'std', 'stroke', 'style', 'sub', 'subtract', 'sum', 'swapaxes', 'swaplevel', 'symboling', 'tail', 'take', 'to_clipboard', 'to_csv', 'to_dense', 'to_dict', 'to_excel', 'to_feather', 'to_gbq', 'to_hdf', 'to_html', 'to_json', 'to_latex', 'to_msgpack', 'to_panel', 'to_period', 'to_pickle', 'to_records', 'to_sparse', 'to_sql', 'to_stata', 'to_string', 'to_timestamp', 'to_xarray', 'transform', 'transpose', 'truediv', 'truncate', 'tshift', 'tz_convert', 'tz_localize', 'unstack', 'update', 'values', 'var', 'where', 'width', 'xs']
    \ 試著把 dataFrame 存放成 excel 用來觀察它
    OK df :> to_excel py: help(pop()) \ 它要求 excel writer 搞不定
    OK df :> to_csv("imports-85.csv") \ csv 很簡單，一下就成功了。

    __main__ :> names constant names // ( -- list ) dataFrame column names
    __main__ :> dtypes constant dtypes // ( -- dict ) dataFrame column dtypes
    __main__ :> names . cr
    ['symboling', 'normalized-losses', 'make', ..snip..., 'price']

    OK __main__ :> dtypes . cr
    {'symboling': <class 'numpy.int32'>, 'normalized-losses': <class 'numpy.float32'>, 'make': <class 'str'>, 'fuel-type': <class 'str'>, 'aspiration': <class 'str'>, 'num-of-doors': <class 'str'>, 'body-style': <class 'str'>, 'drive-wheels': <class 'str'>, 'engine-location': <class 'str'>, 'wheel-base': <class 'numpy.float32'>, 'length': <class 'numpy.float32'>, 'width': <class 'numpy.float32'>, 'height': <class 'numpy.float32'>, 'curb-weight': <class 'numpy.float32'>, 'engine-type': <class 'str'>, 'num-of-cylinders': <class 'str'>, 'engine-size': <class 'numpy.float32'>, 'fuel-system': <class 'str'>, 'bore': <class 'numpy.float32'>, 'stroke': <class 'numpy.float32'>, 'compression-ratio': <class 'numpy.float32'>, 'horsepower': <class 'numpy.float32'>, 'peak-rpm': <class 'numpy.float32'>, 'city-mpg': <class 'numpy.float32'>, 'highway-mpg': <class 'numpy.float32'>, 'price': <class 'numpy.float32'>}

    # Fill missing values in continuous columns with zeros instead of NaN.
    float_columns = [k for k,v in dtypes.items() if v == np.float32]
    df[float_columns] = df[float_columns].fillna(value=0., axis='columns')

    # Fill missing values in continuous columns with '' instead of NaN (NaN mixed with strings is very bad for us).
    string_columns = [k for k,v in dtypes.items() if v == str]
    df[string_columns] = df[string_columns].fillna(value='', axis='columns')

    OK __main__ :> float_columns constant float_columns 
        // ( -- list ) df float_columns
    OK float_columns . cr
    ['normalized-losses', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-size', 'bore', 'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
    OK 

    __main__ :> dnnregressor constant dnnregressor // ( -- obj ) 
    OK dnnregressor . cr
    DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000001E7E70A1748>, 'hidden_units': [50, 30, 10], 'feature_columns': (_NumericColum ... snip .... 

    dnnregressor dir . cr
    ['_Config', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_call_model_fn', '_check_inputs', '_config', '_device_fn', '_evaluate_model', '_extract_metric_update_ops', '_feature_columns', '_feature_engineering_fn', '_features_info', '_filter_predictions', '_get_eval_ops', '_get_feature_ops_from_example', '_get_features_from_input_fn', '_get_predict_ops', '_get_train_ops', '_graph', '_infer_model', '_is_input_constant', '_labels_info', '_model_dir', '_model_fn', '_predict_generator', '_session_config', '_train_model', 'config', 'evaluate', 'export', 'export_savedmodel', 'fit', 'get_params', 'get_variable_names', 'get_variable_value', 'model_dir', 'params', 'partial_fit', 'predict', 'predict_scores', 'set_params']
    OK 

    \ 抓出一筆 feature 
    OK eval_data :> [10:11] . cr
         symboling  normalized-losses    make fuel-type aspiration num-of-doors  \
    174         -1               65.0  toyota    diesel      turbo         four   

        body-style drive-wheels engine-location  wheel-base     ...       \
    174      sedan          fwd           front  102.400002     ...        

         num-of-cylinders  engine-size  fuel-system  bore stroke  \
    174              four        110.0          idi  3.27   3.35   

        compression-ratio  horsepower peak-rpm  city-mpg  highway-mpg  
    174              22.5        73.0   4500.0      30.0         33.0  

    [1 rows x 25 columns]
    OK 
    dnnregressor :> predict(v('eval_data')[0:1]) 

In [ ]:
scores = dnnregressor.predict(x=dict(eval_data[0:10]))

In [ ]:
[i for i in scores]

In [ ]:
eval_data[0:10]

In [ ]:
_ = dnnregressor.predict(x=dict(eval_data[20:21]))
print(eval_data[20:21])
[i for i in _]

In [ ]:
_ = dnnregressor.predict(x=dict(eval_data[27:28]))
print(eval_data[27:28])
[i for i in _]


    OK __main__ :> regressor . cr
    LinearRegressor(
        params = {
            'head' :  < tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x000002025CBB29B0 > ,
            'feature_columns' : [
                _NumericColumn(key = 'symboling', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'normalized-losses', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _HashedCategoricalColumn(key = 'make', hash_bucket_size = 50, dtype = tf.string), 
                _VocabularyListCategoricalColumn(key = 'fuel-type', vocabulary_list = ('diesel', 'gas'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _VocabularyListCategoricalColumn(key = 'aspiration', vocabulary_list = ('std', 'turbo'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _VocabularyListCategoricalColumn(key = 'num-of-doors', vocabulary_list = ('two', 'four'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _VocabularyListCategoricalColumn(key = 'body-style', vocabulary_list = ('hardtop', 'wagon', 'sedan', 'hatchback', 'convertible'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _VocabularyListCategoricalColumn(key = 'drive-wheels', vocabulary_list = ('4wd', 'rwd', 'fwd'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _VocabularyListCategoricalColumn(key = 'engine-location', vocabulary_list = ('front', 'rear'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _NumericColumn(key = 'wheel-base', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'length', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'width', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'height', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'curb-weight', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _VocabularyListCategoricalColumn(key = 'engine-type', vocabulary_list = ('dohc', 'dohcv', 'l', 'ohc', 'ohcf', 'ohcv', 'rotor'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _VocabularyListCategoricalColumn(key = 'num-of-cylinders', vocabulary_list = ('eight', 'five', 'four', 'six', 'three', 'twelve', 'two'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _NumericColumn(key = 'engine-size', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _VocabularyListCategoricalColumn(key = 'fuel-system', vocabulary_list = ('1bbl', '2bbl', '4bbl', 'idi', 'mfi', 'mpfi', 'spdi', 'spfi'), dtype = tf.string, default_value = -1, num_oov_buckets = 0), 
                _NumericColumn(key = 'bore', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'stroke', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'compression-ratio', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'horsepower', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'peak-rpm', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'city-mpg', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None), 
                _NumericColumn(key = 'highway-mpg', shape = (1, ), default_value = None, dtype = tf.float32, normalizer_fn = None)
                ],
            'optimizer' : None,
            'gradient_clip_norm' : None,
            'joint_weights' : False
        }
    )
    OK
